In [10]:
import datetime
import time
import urllib.request
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt

In [11]:
def get_df_from_yahoo_finance (symbol): 

    end_date = datetime.datetime.now()
    end_date = int(round(end_date.timestamp())) * 1000  #<-- Convert datetime to milliseconds 
    end_date_str = str(end_date)
    end_date_str = end_date_str[:10]  #<-- This removes all but the last 10 millisecond values for the time which is format yahoo wants

    start_date_str = '345427200'
  

    ticker_dataframe = {}
    

    # Build Yahoo finance URL for the data request.
    
    yahoo_finance_url_with_symbol = ('https://query1.finance.yahoo.com/v7/finance/download/' + str(symbol)
                                     + '?period1=' + str(start_date_str) + '&period2=' + str(end_date_str)
                                     + '&interval=1mo&events=history&includeAdjustedClose=true')

    url2 = urllib.request.urlopen(yahoo_finance_url_with_symbol) #request data (comes as CSV)
    
    ticker_dataframe = pd.read_csv(url2)  #convert CSV in memory from the Yahoo request to dataframe
    ticker_dataframe.insert(1, "Ticker", symbol)

    return (ticker_dataframe)

In [12]:
list_of_tickers_for_yahoo_data_dump = ['BTC-USD','VTI','SPY','XLF','XLU','XLV','VAW','VNQ','XLP','XLY','FXN','XLI','IYC','QQQ','BND','DJP','GLD','SHV']   # <-- Add tickers here

tickers_and_data_df = pd.DataFrame({})

for ticker in list_of_tickers_for_yahoo_data_dump:
    
    single_ticker_df = get_df_from_yahoo_finance(ticker)
    tickers_and_data_df = pd.concat([tickers_and_data_df, single_ticker_df])


In [13]:
#import fed rates on long term bonds (10 year)
long_term_fed_rates_df = pd.read_csv(                    
    'https://www.econdb.com/api/series/Y10YDUS/?format=csv',                    
    index_col='Date', parse_dates=['Date'])

#import consumer price index (inflation)
cpi_df = pd.read_csv(                    
    'https://www.econdb.com/api/series/CPIUS/?format=csv',                    
    index_col='Date', parse_dates=['Date'])

#import consumer confidence index
consumer_confidence_df = pd.read_csv(                    
    'https://www.econdb.com/api/series/CONFUS/?format=csv',                    
    index_col='Date', parse_dates=['Date'])

#import M2 supply (all money circulating and in bank accounts)
money_supply_df = pd.read_csv(                    
'https://www.econdb.com/api/series/M3US/?format=csv',                    
index_col='Date', parse_dates=['Date'])

In [14]:
#reset index
cpi_df= cpi_df.reset_index()
consumer_confidence_df =consumer_confidence_df.reset_index()
long_term_fed_rates_df= long_term_fed_rates_df.reset_index()
money_supply_df= money_supply_df.reset_index()

In [15]:
#convert date to datetime
cpi_df['Date'] = pd.to_datetime(cpi_df['Date'])
consumer_confidence_df['Date'] = pd.to_datetime(consumer_confidence_df['Date'])
long_term_fed_rates_df['Date'] = pd.to_datetime(long_term_fed_rates_df['Date'])
money_supply_df['Date'] = pd.to_datetime(money_supply_df['Date'])

In [16]:
#drop time from datetime format
cpi_df['Date'] = cpi_df['Date'].dt.date
consumer_confidence_df['Date'] = consumer_confidence_df['Date'].dt.date
long_term_fed_rates_df['Date'] = long_term_fed_rates_df['Date'].dt.date
money_supply_df['Date'] = money_supply_df['Date'].dt.date

In [17]:
#add columns to imitate yahoo df
cpi_df['Ticker'] = 'CPI'
cpi_df['Open'] = cpi_df['CPIUS']
cpi_df['High'] =cpi_df['CPIUS']
cpi_df['Low'] = cpi_df['CPIUS']
cpi_df['Close'] =cpi_df['CPIUS']
cpi_df['Adj Close'] = cpi_df['CPIUS']
cpi_df['Volume'] =cpi_df['CPIUS']

consumer_confidence_df['Ticker'] = 'CC'
consumer_confidence_df['Open'] = consumer_confidence_df['CONFUS']
consumer_confidence_df['High'] =consumer_confidence_df['CONFUS']
consumer_confidence_df['Low'] = consumer_confidence_df['CONFUS']
consumer_confidence_df['Close'] =consumer_confidence_df['CONFUS']
consumer_confidence_df['Adj Close'] = consumer_confidence_df['CONFUS']
consumer_confidence_df['Volume'] =consumer_confidence_df['CONFUS']

long_term_fed_rates_df['Ticker'] = 'Fed_Rates'
long_term_fed_rates_df['Open'] = long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['High'] =long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Low'] = long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Close'] =long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Adj Close'] = long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Volume'] =long_term_fed_rates_df['Y10YDUS']

money_supply_df['Ticker'] = 'M3'
money_supply_df['Open'] = money_supply_df['M3US']
money_supply_df['High'] =money_supply_df['M3US']
money_supply_df['Low'] = money_supply_df['M3US']
money_supply_df['Close'] =money_supply_df['M3US']
money_supply_df['Adj Close'] = money_supply_df['M3US']
money_supply_df['Volume'] =money_supply_df['M3US']



In [18]:
#concatenate and convert all date column to datetime (again becuase it needs to be reminded)
test_df = pd.concat([tickers_and_data_df, cpi_df,consumer_confidence_df,long_term_fed_rates_df,money_supply_df], axis=0)
test_df.drop("CPIUS", axis=1, inplace=True)
test_df['Date']=pd.to_datetime(test_df['Date'])

In [19]:
#create pivot table
pivot = pd.pivot_table(
    data=test_df,
    index='Date',
    columns ='Ticker',
    
)

pivot =pivot.dropna()

In [20]:
# drop all columns except close
pivot = pivot['Close']
pivot

Ticker,BND,BTC-USD,CC,CPI,DJP,FXN,Fed_Rates,GLD,IYC,M3,...,SPY,VAW,VNQ,VTI,XLF,XLI,XLP,XLU,XLV,XLY
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-01,82.360001,338.321014,93.50,237.4,34.060001,24.200001,2.30,112.660004,31.905001,11539.0,...,201.660004,108.400002,78.989998,104.019997,19.366369,55.209999,46.709999,45.470001,67.269997,68.099998
2014-11-01,82.870003,378.046997,95.54,237.0,32.619999,21.500000,2.33,112.110001,33.927502,11615.0,...,207.199997,109.559998,80.570000,106.599998,19.821283,56.910000,49.299999,46.000000,69.610001,71.809998
2014-12-01,82.370003,320.192993,100.70,236.3,29.910000,20.930000,2.21,113.580002,34.439999,11760.0,...,205.539993,107.389999,81.000000,106.000000,20.089357,56.580002,48.490002,47.220001,68.379997,72.150002
2015-01-01,84.349998,217.464005,105.60,234.7,28.570000,20.010000,1.88,123.449997,33.730000,11759.0,...,199.449997,104.480003,86.550003,103.099998,18.692120,54.570000,48.020000,48.320000,69.269997,69.989998
2015-02-01,83.080002,254.263000,102.60,235.3,29.370001,21.879999,1.98,116.160004,36.115002,11847.0,...,210.660004,112.580002,83.370003,109.019997,19.780666,57.490002,50.009998,45.230000,72.239998,75.970001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01,83.000000,38483.125000,72.30,281.9,31.620001,13.460000,1.76,168.089996,75.860001,21646.0,...,449.910004,183.009995,106.239998,226.809998,39.060001,100.739998,75.970001,69.250000,131.229996,184.949997
2022-02-01,81.919998,43193.234375,67.57,284.2,33.939999,14.370000,1.93,178.380005,73.570000,21632.0,...,436.630005,184.130005,102.540001,221.169998,38.520000,99.889999,74.900002,67.930000,129.960007,177.429993
2022-03-01,79.540001,45538.675781,63.91,287.7,36.910000,15.930000,2.13,180.649994,75.040001,21926.0,...,451.640015,194.050003,108.370003,227.669998,38.320000,102.980003,75.889999,74.459999,136.990005,185.000000
